In [1]:
# Test Openai api with langchain is working
from langchain_openai import ChatOpenAI
import os

OpenAI_llm = ChatOpenAI(model="gpt-4o", temperature=0.6, api_key= os.environ['OPENAI_API_KEY'])
messages=[("Say hello if you are working")]
ai_msg = OpenAI_llm.invoke(messages)
ai_msg

AIMessage(content="Hello! I'm here and ready to help. What can I do for you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-3472316d-22a3-496c-8c97-a2fae56a3cc4-0', usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# Testing agents with langchain
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["serpapi"], llm = OpenAI_llm)
agent = initialize_agent(tools, llm=OpenAI_llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)

agent.run("What is current value of 1 dollar in rupees?")

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_9244\1254470714.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm=OpenAI_llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)
C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_9244\1254470714.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~i

'The current value of 1 US dollar is approximately 86.53 Indian Rupees.'

In [12]:
from langchain_openai import ChatOpenAI
from config import constants
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.6,
    api_key= constants.OPENAI_API_KEY
)

In [2]:
from langchain.chains.conversation.memory import ConversationBufferMemory 
# initialize conversational memory 

conversational_memory = ConversationBufferMemory(
    memory_key='chat_history',
    k=5,
    return_messages = True
)

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_10136\3732751371.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferMemory(


In [14]:
from langchain.tools import BaseTool
import requests
import json
class GetClientDetails(BaseTool):
    name: str = "Client details extractor"
    description: str = "use this tool when you have given a platform name, idea or platform link and you want to find more information about CEO of platform and proposed idea"
    def _run(self, website):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "sonar-pro",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are artificial intelligence agent which extracts an information by searching about it online"
                        "and returns information strictly in the following structured JSON format if it exists:"
                        "{"
                        "  'Title of website or platform': <title_of_platform>,"
                        "  'Proposed Idea': <proposed_idea>,"
                        "  'CEO Info': <ceo_info>"
                        "}"
                        "Please make sure the response has proper line breaks, no extra spaces, unexpected characters (like \\n, extra quotes), or non-escaped characters."
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "I am giving you platform name,idea or platform link and you have to find latest information about" 
                        "the platform by going to given link and return atleast 200 words description about idea and"
                        "atleast 150 words description about CEO, their goal and achievements"
                        "Please try to make it as detailed as you can and always refer to online information"
                        "here is the platform link or name "
                        f"{str(website)}"
                    )
                },
            ],
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers).json()
        return  response["choices"][0]["message"]["content"]
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

In [15]:
class GetPlatformInfo(BaseTool):
    name: str = "Platform info extractor"
    description: str = "use this tool when you have given a description about meeting and you have to find the proposed idea, client name or platform link from given description"

    def _run(self, description):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "sonar-pro",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have given a description of meeting and you will have to find out proposed idea, client name and website link mentioned in the given description."
                        "Use client info extractor tool to get more information."
                        "Return it strictly in the following structured JSON format: "
                        "{"
                        "  'Idea': <idea>,"
                        "  'Client Info': <client_info>,"
                        "  'Platform Link': <platform_link>"
                        "}"
                        "Please make sure the response has proper line breaks, no extra spaces, unexpected characters (like \\n, extra quotes), or non-escaped characters."
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "if anything is not provided in description then use internet and google search to find out the actual information"
                        "Here is the meeting description " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers).json()
        return response["choices"][0]["message"]["content"]
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

In [16]:
class GetIdeaSolution(BaseTool):
    name: str = "Solution extractor"
    description: str = "use this tool when you have given an idea description and information about client and you have to find solution on how we can achieve the given idea"

    def _run(self, description):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "sonar-pro",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have given an idea description and you have to find around 3-4 solutions to achieve the given idea using AI-ML"
                        "or webdev technologies. Search online about tech stack, resources, timeline of project and youtube videos and send it with proper formatting strictly in the following structured JSON format with proper line breaks, no extra spaces, unexpected characters (e.g., \\n, extra quotes, unescaped characters), and proper escaping. "
                        "Ensure the solutions are wrapped within a key \"solutions\", and each solution has the following format: "
                        "{"
                        "  'Solution': <solution>,"
                        "  'Tech Stack': <tech_stack>,"
                        "  'Timeline': <timeline>"
                        "}"
                        "The response should have a structure similar to the following response:"
                        "{" 
                            "'Solutions': ["
                                "{"
                                "   'Solution': <solution_1>,"
                                "   'Tech Stack': <tech_stack_1>,"
                                "   'Timeline': <timeline_1>"
                                "},"
                                "{"
                                "   'Solution': <solution_2>,"
                                "   'Tech Stack': <tech_stack_2>,"
                                "   'Timeline': <timeline_2>"
                                "},"
                                "{"
                                "   'Solution': <solution_3>,"
                                "   'Tech Stack': <tech_stack_3>,"
                                "   'Timeline': <timeline_3>"
                                "},"
                                "{"
                                "   'Solution': <solution_4>,"
                                "   'Tech Stack': <tech_stack_4>,"
                                "   'Timeline': <timeline_4>" 
                                "}"    
                            "]"
                        "}"  
                        
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "Here is the idea description: " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers).json()
        return response["choices"][0]["message"]["content"]
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

In [17]:
# Creating Agent
from langchain.agents import initialize_agent
tools = [GetClientDetails(), GetPlatformInfo(), GetIdeaSolution()]
agent = initialize_agent(
    tools=tools, 
    llm = llm, 
    agent="zero-shot-react-description", 
    verbose=True, 
    max_iterations=3, 
    early_stopping_method='generate', 
    memory = conversational_memory, 
    handle_parsing_errors=True
)

In [18]:
para = agent.run(
      "First find the idea, client name and platform link using Platform info extractor tool and find 150 words information about proposed idea about product, goals and achievements of CEO. Once you got the information about their idea then find how we can help them to build it in 200 words "
      "fetch realtime data from internet everytime."
      "This information is going to be added in project proposal so please write in detail and sections like CEO_Info,idea and solution must be explained in 600 words each. "
      "Return your response in structured JSON format like below and please use proper line breaks: "
      """
      {
      "platform_name": <platform_name>,
      "CEO_Info": <ceo_info>,
      "CEO_Name": <ceo_name>,
      "idea": <idea>,
      "solution": <solution>,
      "tech_stack": <tech stack>,
      "timeline": <timeline>,
      "Platform_link": <platform_link>
      }
      """
      "Here is the description: "
      "Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc"
      )
output = json.loads(para)
print(output)



> Entering new AgentExecutor chain...
To answer the question, I need to first extract the proposed idea, client name, and platform link from the provided description. 

Action: Platform info extractor
Action Input: "Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc"
Observation: {
  "Idea": "Making a Twitter automation tool with features like automated tweets, scheduled tweets, and tweet improvement guides",
  "Client Info": "Rohan",
  "Platform Link": null
}
Thought:I have extracted the idea and client name. However, the platform link is not available. Next, I need to find information about the CEO, which in this case is Rohan, and details about the proposed idea. 

Action: Client details extractor
Action Input: "Making a Twitter automation tool with features like automated tweets, scheduled tweets, and tweet improvement guide

OutputParserException: Could not parse LLM output: ````json
{
  "platform_name": "Twitter Automation Tools",
  "CEO_Info": "While there isn't a single CEO for all Twitter automation tools, we can look at the leadership of some prominent companies in this space. For example, Emeric Ernoult is the CEO and co-founder of Agorapulse, a popular social media management tool. Ernoult's goal has been to simplify social media management for businesses and agencies. Under his leadership, Agorapulse has grown to serve over 31,000 social media managers worldwide. Justyn Howard is the CEO and co-founder of Sprout Social, another leading social media management platform. Howard's vision has been to create tools that make social media marketing more effective and data-driven. Since founding Sprout Social in 2010, he has led the company to become a publicly-traded entity, serving over 30,000 brands and organizations globally. These leaders share common goals of innovating in the social media management